# Running Monte Carlo Transport Independently

This tutorial demonstrates how to run the Monte Carlo transport loop directly using `Simulation.from_config` without running full TARDIS iterations. This approach gives you direct control over the Monte Carlo transport process.

In [1]:
from pathlib import Path

import astropy.units as u

from tardis.io.atom_data import AtomData
from tardis.io.configuration.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.transport.montecarlo.estimators.radfield_mc_estimators import (
    initialize_estimator_statistics,
)
from tardis.transport.montecarlo.montecarlo_main_loop import (
    montecarlo_main_loop,
)
from tardis.transport.montecarlo.packets.trackers import (
    generate_rpacket_last_interaction_tracker_list,
    generate_rpacket_tracker_list,
    rpacket_last_interaction_tracker_list_to_dataframe,
)

/Users/wkerzend/python/tardis/tardis/__init__.py:17: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

Initializing tabulator and plotly panel extensions for widgets to work


In [2]:
# User-configurable variables
CONFIG_FILE_NAME = "tardis_example.yml"
NUMBER_OF_PACKETS = 10000
NUMBER_OF_VPACKETS = 0  # Set to 0 to disable virtual packets
ITERATION_NUMBER = 1
SHOW_PROGRESS_BARS = True
TOTAL_ITERATIONS = 1
ENABLE_RPACKET_TRACKING = False  # True: full tracking, False: last interaction only

In [3]:
# Setup simulation state from config
config_file = Path(CONFIG_FILE_NAME)
if not config_file.exists():
    raise FileNotFoundError(f"Configuration file {CONFIG_FILE_NAME} not found")

config = Configuration.from_yaml(str(config_file))
atom_data = AtomData.from_hdf("kurucz_cd23_chianti_H_He_latest.h5")
sim = Simulation.from_config(config, atom_data=atom_data)

print("Simulation created successfully!")

WARNING	Task(Task-3) tardis.io.model.parse_composition_configuration:parse_composition_configuration.py:parse_density_from_config()- Number of density points larger than number of shells. Assuming inner point irrelevant
WARNING	Task(Task-3) tardis.io.model.parse_mass_fraction_configuration:parse_mass_fraction_configuration.py:parse_mass_fractions_from_config()- model_isotope_time_0 is not set in the configuration. Isotopic mass fractions will not be decayed and is assumed to be correct for the time_explosion. THIS IS NOT RECOMMENDED!
WARNING	Task(Task-3) tardis.io.model.parse_mass_fraction_configuration:parse_mass_fraction_configuration.py:parse_mass_fractions_from_config()- model_isotope_time_0 is not set in the configuration. Isotopic mass fractions will not be decayed and is assumed to be correct for the time_explosion. THIS IS NOT RECOMMENDED!
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine depr

Simulation created successfully!


In [4]:
# Initialize opacity and macro atom states manually
sim.opacity_state = sim.opacity.legacy_solve(sim.plasma)

if sim.macro_atom is not None:
    sim.macro_atom_state = sim.macro_atom.solve(
        sim.plasma.j_blues,
        sim.plasma.atomic_data,
        sim.opacity_state.tau_sobolev,
        sim.plasma.stimulated_emission_factor,
        sim.opacity_state.beta_sobolev,
    )
else:
    sim.macro_atom_state = None

print("Opacity and macro atom states initialized!")

Opacity and macro atom states initialized!


In [29]:
# Extract states from simulation
geometry_state = sim.simulation_state.geometry
opacity_state = sim.opacity_state
montecarlo_configuration = sim.transport.montecarlo_configuration
time_explosion = sim.simulation_state.time_explosion.to(u.s).value
spectrum_frequency_grid = sim.transport.spectrum_frequency_grid.to(u.Hz).value
packet_source = sim.transport.packet_source

# Initialize estimators
tau_sobolev_shape = opacity_state.tau_sobolev.shape
gamma_shape = (0, geometry_state.no_of_shells)
estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)

# Convert to numba-compatible versions
geometry_state_numba = geometry_state.to_numba()
line_interaction_type = montecarlo_configuration.LINE_INTERACTION_TYPE
opacity_state_numba = opacity_state.to_numba(sim.macro_atom_state, line_interaction_type)

print("Monte Carlo states prepared!")

Monte Carlo states prepared!


In [35]:
# Create packet collection
seed_offset = montecarlo_configuration.PACKET_SEEDS
packet_collection = packet_source.create_packets(NUMBER_OF_PACKETS, seed_offset)
ENABLE_RPACKET_TRACKING = True
# Setup packet tracking
if ENABLE_RPACKET_TRACKING:
    rpacket_trackers = generate_rpacket_tracker_list(
        NUMBER_OF_PACKETS,
        montecarlo_configuration.INITIAL_TRACKING_ARRAY_LENGTH,
    )
    rpacket_tracker_collection = None
    print("Using full RPacket tracking")
else:
    # Initialize the last interaction tracker collection
    # Generate individual trackers for the main loop
    rpacket_trackers = generate_rpacket_last_interaction_tracker_list(
        NUMBER_OF_PACKETS
    )
    print("Using last interaction tracking with collection")

Using full RPacket tracking


In [36]:
# Run the Monte Carlo main loop
v_packets_energy_hist, last_interaction_tracker, vpacket_tracker = (
    montecarlo_main_loop(
        packet_collection,
        geometry_state_numba,
        time_explosion,
        opacity_state_numba,
        montecarlo_configuration,
        estimators,
        spectrum_frequency_grid,
        rpacket_trackers,
        NUMBER_OF_VPACKETS,
        SHOW_PROGRESS_BARS,
    )
)


In [37]:
# Create DataFrame from tracker data
if ENABLE_RPACKET_TRACKING:
    # Full tracking: convert from rpacket_trackers list
    from tardis.transport.montecarlo.packets.trackers import rpacket_trackers_to_dataframe
    tracker_df = rpacket_trackers_to_dataframe(rpacket_trackers)
    print(f"Full tracking DataFrame created: {tracker_df.shape}")
else:
    # Last interaction tracking: convert from rpacket_trackers list  
    tracker_df = rpacket_last_interaction_tracker_list_to_dataframe(rpacket_trackers)
    print(f"Last interaction DataFrame created: {tracker_df.shape}")

print("Available columns:", list(tracker_df.columns))

Full tracking DataFrame created: (340470, 22)
Available columns: ['status', 'seed', 'r', 'nu', 'mu', 'energy', 'shell_id', 'interaction_type', 'line_before_nu', 'line_before_mu', 'line_before_id', 'line_after_nu', 'line_after_mu', 'line_after_id', 'escat_before_mu', 'escat_after_mu', 'continuum_before_nu', 'continuum_before_energy', 'continuum_before_mu', 'continuum_after_nu', 'continuum_after_energy', 'continuum_after_mu']


In [38]:
last_tracker_df

,last_interaction_type,last_line_interaction_in_id,last_line_interaction_in_nu,last_line_interaction_out_id
packet_id,,,,
0,ESCATTERING,-1,5.150244e+14,-1
1,ESCATTERING,-1,1.031473e+15,-1
2,ESCATTERING,-1,1.188678e+15,-1
3,ESCATTERING,-1,7.361418e+14,-1
4,NO_INTERACTION,-1,2.763790e+14,-1
...,...,...,...,...
9995,NO_INTERACTION,-1,4.272588e+14,-1
9996,NO_INTERACTION,-1,1.888737e+14,-1
9997,NO_INTERACTION,-1,3.801189e+14,-1


In [ ]:
def convert_to_last_interaction_dataframe(full_tracker_df):
    """
    Convert full interaction DataFrame to last interaction DataFrame.
    
    Matches the behavior of RPacketLastInteractionTracker which captures
    the final packet state regardless of interaction type.
    
    Parameters
    ----------
    full_tracker_df : pandas.DataFrame
        Full tracking DataFrame with multi-index (packet_id, step)
        
    Returns
    -------
    pandas.DataFrame
        Last interaction DataFrame with one row per packet
    """
    import pandas as pd
    from tardis.transport.montecarlo.packets.radiative_packet import InteractionType
    from pandas.api.types import CategoricalDtype
    
    # Reset index to work with packet_id and step as columns
    df = full_tracker_df.reset_index()
    
    last_interactions = []
    
    for packet_id in df['packet_id'].unique():
        # Get all interactions for this packet
        packet_data = df[df['packet_id'] == packet_id]
        
        if len(packet_data) == 0:
            continue
            
        # Get the very last row (final state) - this matches last interaction tracker behavior
        last_row = packet_data.iloc[-1].copy()
        
        # Create the last interaction row matching the expected format exactly
        # Only include the columns that the native last interaction tracker produces
        last_interaction_row = {
            'packet_id': packet_id,
            'last_interaction_type': last_row['interaction_type'],
            'last_line_interaction_in_id': last_row.get('line_interaction_in_id', -1),
            'last_line_interaction_in_nu': last_row['nu'],  # Use current frequency as input frequency
            'last_line_interaction_out_id': last_row.get('line_interaction_out_id', -1),
        }
        
        # Fix line interaction handling - if not a line interaction, set IDs to -1
        if last_row['interaction_type'] != InteractionType.LINE:
            last_interaction_row['last_line_interaction_in_id'] = -1
            last_interaction_row['last_line_interaction_out_id'] = -1
            
        last_interactions.append(last_interaction_row)
    
    # Create DataFrame from collected last interactions
    last_interaction_df = pd.DataFrame(last_interactions)
    last_interaction_df.set_index('packet_id', inplace=True)
    
    # Create categorical dtype matching the native implementation
    interaction_type_dtype = CategoricalDtype(
        categories=[member.name for member in InteractionType],
        ordered=False
    )
    
    # Convert interaction_type to categorical to match native format
    last_interaction_df['last_interaction_type'] = pd.Categorical(
        [InteractionType(last_interaction_df.loc[idx, 'last_interaction_type']).name 
         for idx in last_interaction_df.index],
        dtype=interaction_type_dtype
    )
    
    return last_interaction_df

# Test the function if we have full tracking data
if ENABLE_RPACKET_TRACKING and 'tracker_df' in locals():
    print("Converting full tracking DataFrame to last interaction format...")
    last_interaction_df = convert_to_last_interaction_dataframe(tracker_df)
    print(f"Last interaction DataFrame shape: {last_interaction_df.shape}")
    print("Last interaction DataFrame columns:", list(last_interaction_df.columns))
    print("\nSample last interactions:")
    print(last_interaction_df.head())
else:
    print("Function defined. Set ENABLE_RPACKET_TRACKING=True to test conversion.")

Converting full tracking DataFrame to last interaction format...
Last interaction DataFrame shape: (10000, 8)
Last interaction DataFrame columns: ['last_interaction_type', 'last_r', 'last_nu', 'last_mu', 'last_energy', 'last_shell_id', 'last_line_interaction_in_id', 'last_line_interaction_out_id']

Sample last interactions:
          last_interaction_type  last_r  last_nu  last_mu  last_energy  \
packet_id                                                                
0                NO_INTERACTION     0.0      0.0      0.0          0.0   
1                NO_INTERACTION     0.0      0.0      0.0          0.0   
2                NO_INTERACTION     0.0      0.0      0.0          0.0   
3                NO_INTERACTION     0.0      0.0      0.0          0.0   
4                NO_INTERACTION     0.0      0.0      0.0          0.0   

           last_shell_id  last_line_interaction_in_id  \
packet_id                                               
0                      0                 

In [40]:
last_interaction_df

,last_interaction_type,last_r,last_nu,last_mu,last_energy,last_shell_id,last_line_interaction_in_id,last_line_interaction_out_id
packet_id,,,,,,,,
0,NO_INTERACTION,0.0,0.0,0.0,0.0,0,-1,-1
1,NO_INTERACTION,0.0,0.0,0.0,0.0,0,-1,-1
2,NO_INTERACTION,0.0,0.0,0.0,0.0,0,-1,-1
3,NO_INTERACTION,0.0,0.0,0.0,0.0,0,-1,-1
4,NO_INTERACTION,0.0,0.0,0.0,0.0,0,-1,-1
...,...,...,...,...,...,...,...,...
9995,NO_INTERACTION,0.0,0.0,0.0,0.0,0,-1,-1
9996,NO_INTERACTION,0.0,0.0,0.0,0.0,0,-1,-1
9997,NO_INTERACTION,0.0,0.0,0.0,0.0,0,-1,-1
